In [1]:
import sys
sys.path.insert(0, '../Lab/')
sys.path.insert(0, '../New_Hybrids/')
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from UserIcmKNNCFRecommender import UserIcmKNNCFRecommender
from ItemIcmKNNCFRecommender import ItemIcmKNNCFRecommender
from WeightedHybridV2 import WeightedHybridScoreRecommender
from optimize_weights import optimize_weights
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
import matplotlib
import matplotlib.pyplot as plt
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python

%matplotlib inline 

import numpy as np
sys.path.insert(0, '../datasets/')
from load_tables import load_ICM_csr, load_URM_csr, load_clustered_ICM_csr
def plotta(t,maps):
    fig, ax = plt.subplots()
    ax.plot(t, maps)

    ax.set(xlabel='T', ylabel='MAP')
    ax.grid()
    plt.show()

In [2]:

from ParameterTuning.SearchBayesianSkoptCross import SearchBayesianSkoptCross
from ParameterTuning.SearchAbstractClassCross import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical

In [3]:
#ICM_all = load_ICM_csr()
#ICM_cl = load_clustered_ICM_csr()
URM_all = load_URM_csr()

In [4]:

#URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.9)
#URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.8)

#evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
#evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[ 10])

In [5]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs


def hyperparameter_optimization_CFW_D(URM_train, ICM_train, W_sparse_CF):

    recommender_class = UserKNNCFRecommender

    parameterSearch = SearchBayesianSkoptCross(recommender_class,
                                               n_validations = 2,
                                     evaluator_validation=None,
                                     evaluator_test=None)


    hyperparameters_range_dictionary = {}
    hyperparameters_range_dictionary["topK"] = Integer(5, 1000)
    hyperparameters_range_dictionary["shrink"] = Integer(0, 100)

    #hyperparameters_range_dictionary["add_zeros_quota"] = Real(low = 0, high = 1, prior = 'uniform')
    #hyperparameters_range_dictionary["normalize_similarity"] = Categorical([True, False])


    recommender_input_args = SearchInputRecommenderArgs(
        CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],
        CONSTRUCTOR_KEYWORD_ARGS = {},
        FIT_POSITIONAL_ARGS = [],
        FIT_KEYWORD_ARGS = {}
    )


    output_folder_path = "result_experiments/"

    import os

    # If directory does not exist, create
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    n_cases = 100
    metric_to_optimize = "MAP"


    # Clone data structure to perform the fitting with the best hyperparameters on train + validation data
    recommender_input_args_last_test = recommender_input_args.copy()
    recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS[0] = URM_train


    parameterSearch.search(recommender_input_args,
                           recommender_input_args_last_test = recommender_input_args_last_test,
                           parameter_search_space = hyperparameters_range_dictionary,
                           n_cases = n_cases,
                           n_random_starts = int(n_cases/3),
                           save_model = "no",
                           output_folder_path = output_folder_path,
                           output_file_name_root = recommender_class.RECOMMENDER_NAME,
                           metric_to_optimize = metric_to_optimize
                          )

In [ ]:
hyperparameter_optimization_CFW_D(URM_all, None, None)

Iteration No: 1 started. Evaluating function at random point.
UserKNNCFRecommender: URM Detected 53 (0.67 %) cold users.
UserKNNCFRecommender: URM Detected 2356 (9.07 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 790, 'shrink': 22}
Similarity column 7947 ( 100 % ), 2254.97 column/sec, elapsed time 0.06 min
EvaluatorHoldout: Processed 5625 ( 100.00% ) in 11.93 sec. Users per second: 471
UserKNNCFRecommender: URM Detected 60 (0.76 %) cold users.
UserKNNCFRecommender: URM Detected 2302 (8.86 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 790, 'shrink': 22}
Similarity column 7947 ( 100 % ), 5523.89 column/sec, elapsed time 0.02 min
EvaluatorHoldout: Processed 5595 ( 100.00% ) in 9.54 sec. Users per second: 587
UserKNNCFRecommender: URM Detected 60 (0.76 %) cold users.
UserKNNCFRecommender: URM Detected 2302 (8.86 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 790, 'shrink': 22}
Similarity column 7947 ( 100 % ), 5460.16 column/sec, e

In [ ]:
#Config 14: {'topK': 52, 'add_zeros_quota': 0.8588004964327149, 'normalize_similarity': False} itemknn